In [1]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

Mounted at /content/drive


In [2]:
!pip install pytorch-lightning
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 32.1 MB/s 
     |████████████████████████████████| 125 kB 72.3 MB/s 
     |████████████████████████████████| 529 kB 67.2 MB/s 
     |████████████████████████████████| 87 kB 7.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=bff167ad8379a208fc0ff4b2bfd05f49963151130e449a8346550887a576719d
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 36.6 MB/s 
     |████████████████████████████████| 182 kB 69.2 MB/s 


In [3]:
import numpy as np
import pandas as pd

import os
from tqdm import tqdm
import time

import cv2
import torch
import torch.nn as nn

import pytorch_lightning as pl
import timm

In [4]:
ROOT_DIR = "/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection"
GROUP_DIR = f"{ROOT_DIR}/data/Zalo22/LivenessDetection"

os.chdir(ROOT_DIR)

In [5]:
model_name = "2D_singleframe_tf_efficientnetv2_s_in21ft1k"
version = "size384_cropface"

checkpoint_dir = f"{ROOT_DIR}/checkpoints/{model_name}/{version}" # chỗ chứa weight.ckpt và opt.json
weight_name = "weight.ckpt"
weight_dir = f"{checkpoint_dir}/{weight_name}"

# train_data_dir = f"/content/train" # trỏ đến folder train
# test_data_dir = f"/content/public" # trỏ đến folder public, trong đó public/videos/ chứa các .mp4
test_data_dir2 = f"/content/public_test_2" # trỏ đến folder public_test_2 trong đó chứa các .mp4
private_test = f"/content/private_test"
# checkpoint_dir:
# weight_dir: trỏ đến weight load vào model. Ví dụ: ./checkpoints/modelABC/weight_1.ckpt

In [6]:
# --- load config file and transform config

import json
class Dict2Class(object):
    def __init__(self, my_dict):
          
        for key in my_dict:
            setattr(self, key, my_dict[key])

cfg = json.load(open(f"{checkpoint_dir}/opt.json")) # f"{checkpoint_dir}/opt.json": trỏ đến vị trí chứa opt.json
cfg = Dict2Class(cfg)

import importlib
transform_cfg = importlib.import_module(f'configs.{cfg.transform_config}').CFG

# Inference

In [7]:
# --- load data to disk ---

import zipfile

with zipfile.ZipFile(f"{ROOT_DIR}/data/private_test.zip", 'r') as zip_ref:
    zip_ref.extractall(f"/content")

# with zipfile.ZipFile(f"{GROUP_DIR}/data/train.zip", 'r') as zip_ref:
#     zip_ref.extractall(f"/content")

# with zipfile.ZipFile(f"{GROUP_DIR}/data/public_test.zip", 'r') as zip_ref:
#     zip_ref.extractall(f"/content")

with zipfile.ZipFile(f"{GROUP_DIR}/data/public_test_2.zip", 'r') as zip_ref:
    zip_ref.extractall(f"/content")

In [8]:
print(len(os.listdir(f"/content/private_test/videos")))

1253


In [9]:
# import sys
# sys.path.append(ROOT_DIR)

from models import LivenessModel2D, LivenessModel2DLSTM
from datasets import LivenessDataset

In [19]:
# LB2

cfg.num_frames = 5
cfg.frame_step = -1

fnames = sorted(os.listdir(f"{test_data_dir2}/videos"), key=lambda x: int(x.split(".")[0]))
test_df = pd.DataFrame({"fname": fnames}).head(1)
test_dataset = LivenessDataset(df=test_df, root_dir=test_data_dir2, transforms=transform_cfg.create_val_transforms(cfg.height, cfg.width), num_frames=cfg.num_frames, ext=cfg.ext, frame_step=cfg.frame_step)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
trainer = pl.Trainer(accelerator='cuda')
preds = trainer.predict(model, test_loader)

preds = torch.cat(preds).detach().cpu().numpy()
test_df["liveness_score"] = preds
test_df.to_csv(f"{checkpoint_dir}/private_test.csv", index = False)

Use 5 frame | Load image from mp4 | Face crop False


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [40]:
# predictions on LB2
cfg.num_frames = 5 # sử dụng 5 frame để infer
cfg.frame_step = -1 # =-1 thì dải đều cả video
cfg.batch_size = 1
device = 'cuda'

fnames = sorted(os.listdir(f"{private_test}/videos"), key=lambda x: int(x.split(".")[0]))
test_df = pd.DataFrame({"fname": fnames})
test_dataset = LivenessDataset(df=test_df, root_dir=private_test, transforms=transform_cfg.create_val_transforms(cfg.height, cfg.width), num_frames=cfg.num_frames, ext=cfg.ext, frame_step=cfg.frame_step)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
model.to(device)
model.eval()
preds = []
durations = []
with torch.no_grad():
    for idx, (img, label) in tqdm(enumerate(test_loader)):
        begin = time.time()

        # inference each image
        img = img.to(device)
        pred = model(img).view(-1).cpu().numpy()
        preds.append(pred)

        duration = time.time() - begin
        durations.append(duration)
        # print(f"Inference time video {test_df.iloc[idx]['fname']}: {duration}")

preds = np.concatenate(preds)
test_df["liveness_score"] = preds
test_df.to_csv(f"{checkpoint_dir}/private_test2.csv", index = False)

Use 5 frame | Load image from mp4 | Face crop False


1253it [03:11,  6.53it/s]


In [ ]:
# predictions on LB1
cfg.num_frames = 5 # sử dụng 5 frame để infer
cfg.frame_step = -1 # =-1 thì dải đều cả video
cfg.batch_size = 1
device = 'cuda'

fnames = sorted(os.listdir(f"{test_data_dir}/videos"), key=lambda x: int(x.split(".")[0]))
test_df = pd.DataFrame({"fname": fnames})
test_dataset = LivenessDataset(df=test_df, root_dir=test_data_dir, transforms=transform_cfg.create_val_transforms(cfg.height, cfg.width), num_frames=cfg.num_frames, ext=cfg.ext)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
model.to(device)

preds = []
durations = []
with torch.no_grad():
    for idx, (img, label) in enumerate(test_loader):
        begin = time.time()

        # inference each image
        img = img.to(device)
        pred = model(img).view(-1).cpu().numpy()
        preds.append(pred)

        duration = time.time() - begin
        durations.append(duration)
        # print(f"Inference time video {test_df.iloc[idx]['fname']}: {duration}")

preds = np.concatenate(preds)

test_df["liveness_score"] = preds
test_df.to_csv(f"{checkpoint_dir}/predict_lb1_5frame_again.csv", index = False)

In [ ]:
# prediction on validation set
cfg.num_frames = 1 # sử dụng 5 frame để infer
cfg.frame_step = -1 # =-1 thì dải đều cả video

data_df = pd.read_csv(f"/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection/data/Zalo22/LivenessDetection/data/meta_data/identified_metadata.csv")
data_df = data_df[data_df.set == "train"]
train_df = data_df[data_df['fold']!=cfg.fold].reset_index(drop=True)
valid_df = data_df[data_df['fold']==cfg.fold].reset_index(drop=True)

valid_dataset = LivenessDataset(df=valid_df, root_dir=train_data_dir, ext=cfg.ext, transforms=transform_cfg.create_val_transforms(cfg.height, cfg.width), num_frames=cfg.num_frames)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
trainer = pl.Trainer()
preds = trainer.predict(model, valid_loader, accelerator='cuda')

preds = torch.cat(preds).detach().cpu().numpy()
valid_df["pred_score"] = preds
valid_df.to_csv(f"{checkpoint_dir}/valid.csv", index = False)